In [1]:
import json
import os
import pandas as pd
from src.RAG_dataset_generator import RAGDatasetGenerator

In [2]:
CHUNKS_FOLDER = "../../data\D26\ladrome-2023-12-22-page-chunked-v1"
FILE_LIST = os.listdir(CHUNKS_FOLDER)

In [3]:
def read_json(path):
    with open(path, "r") as f:
        return json.load(f)

In [9]:
handmade_dataset = pd.read_excel("../../data/D26/D26_dataset_handmade.xlsx")
handmade_dataset["relevant_chunk_ids"] = handmade_dataset["relevant_chunk_ids"].str.split(" ")
handmade_dataset.dropna(subset="relevant_chunk_ids", inplace=True)

filenames = handmade_dataset["relevant_chunk_ids"].tolist()
filenames = [fn[0] for fn in filenames]

dataset_generator = RAGDatasetGenerator("openai", model="gpt-3.5-turbo")
dataset_generator.generate_RAG_dataset(CHUNKS_FOLDER, filenames)

In [24]:
dataset_generator = RAGDatasetGenerator("openai", model="gpt-3.5-turbo")

questions = []
answers = []
short_answers = []
relevant_ids = []
for chunk_ids in reversed(handmade_dataset["relevant_chunk_ids"].tolist()):
    chunk_id = chunk_ids[0]
    chunk = read_json(os.path.join(CHUNKS_FOLDER, chunk_id))
    text = chunk["text"]
    question = dataset_generator.generate_questions_from_chunk(text, 1)[0]
    answer = dataset_generator.generate_answer(question, text)
    short_answer = dataset_generator.generate_answer(question, text, short=True)
    questions.append(question)
    answers.append(answer)
    short_answers.append(short_answer)
    relevant_ids.append(chunk_id)
    break

generated_df = pd.DataFrame.from_dict({"query": questions, "relevant_chunk_ids": relevant_ids, "long_answer": answers, "short_answer": short_answers})
generated_df.to_csv(os.path.join(os.path.dirname(CHUNKS_FOLDER), "generated_dataset.csv"), index=False)


In [10]:
filenames = handmade_dataset["relevant_chunk_ids"].tolist()
filenames = [fn[0] for fn in filenames]

dataset_generator = RAGDatasetGenerator("openai", model="gpt-3.5-turbo")
dataset_generator.generate_RAG_dataset(CHUNKS_FOLDER, filenames)

100%|██████████| 79/79 [06:16<00:00,  4.76s/it]

Dataset saved at : ../../data\D26\generated_data_gpt-3-5-turbo
